# Test benchmark

## Setup

In [1]:
import sys
import subprocess
from antidote import __version__, is_compiled
print(f"""
== Python ==
{sys.version}

== Antidote =
{__version__} {'(cython)' if is_compiled() else ''}
""")


== Python ==
3.9.2 (default, Feb 19 2021, 13:34:46) 
[GCC 9.3.0]

== Antidote =
1.0.0 (cython)



In [2]:
cat /proc/cpuinfo | grep 'model name' | head -n 1

model name	: AMD Ryzen 7 PRO 4750U with Radeon Graphics


## Benchmarks

### Utilities

In [3]:
from antidote import factory, Service, implementation, Factory, world


class DependencyGenerator:
    services = []
    factories = dict()
    implementations = dict()

    @staticmethod
    def make_factory(Output):
        @factory
        def f() -> Output:
            return Output()

        return f

    @staticmethod
    def make_implementation(Interface, Impl):
        @implementation(Interface)
        def f():
            return Impl

        return f

    @staticmethod
    def make_factory_method(Output):
        def f(self) -> Output:
            return Output()

        return f
    
    @classmethod
    def add_service(cls, i: int):
        cls.services.append(type(f"Service{i}", (Service,), {}))
    
    @classmethod
    def add_factory(cls, i: int):
        Output = type(f"Ouput{i}", (object,), {})
        cls.factories[Output] = cls.make_factory(Output)

    @classmethod
    def add_factory_class(cls, i: int):
        Output = type(f"Ouput{i}", (object,), {})
        cls.factories[Output] = type(f"Factory{i}", (Factory,), {"__call__":  cls.make_factory_method(Output)})
        
    @classmethod
    def add_implementation(cls, i: int):
        Interface = type(f"Interface{i}", (object,), {})
        Impl = type(f"Impl{i}", (Interface, Service), {})
        cls.implementations[Interface] =  cls.make_implementation(Interface, Impl)
        
    @classmethod
    def generate_dependencies(cls, n: int):
        k = n // 4
        for i in range(k):
            cls.add_service(i)
            cls.add_factory(i + 1)
            cls.add_factory_class(i + 2)
            cls.add_implementation(i + 3)
            
        # Instantiating all dependencies

        for service in cls.services:
            world.get(service)

        for output, factory in cls.factories.items():
            world.get(output @ factory)

        for interface, impl in cls.implementations.items():
            world.get(interface @ impl)
    
class Benchmark:
    @staticmethod
    def empty():
        with world.test.empty():
            pass

    @staticmethod
    def new():
        with world.test.new():
            pass

    @staticmethod
    def clone():
        with world.test.clone():
            pass

    @staticmethod
    def clone_keeping_singletons():
        with world.test.clone(keep_singletons=True):
            pass
    
    @staticmethod
    def run():
        print("world.test.empty()")
        f = Benchmark.empty
        %timeit f()
        print("\nworld.test.new()")
        f = Benchmark.new
        %timeit f()
        print("\nworld.test.clone()")
        f = Benchmark.clone
        %timeit f()
        print("\nworld.test.clone(keep_singletons=True)")
        f = Benchmark.clone_keeping_singletons
        %timeit f()
        
    

### Isolation
#### Imitating a medium project

Creating **40** different dependencies, all singletons, in all the different ways that actually impact test utilities.

In [4]:
DependencyGenerator.generate_dependencies(n = 40)

In [5]:
Benchmark.run()

world.test.empty()
11 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

world.test.new()
37.2 µs ± 336 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

world.test.clone()
12.4 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

world.test.clone(keep_singletons=True)
12.9 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


#### Imitating a big project

Creating **400** different dependencies, all singletons, in all the different ways that actually impact test utilities.

In [6]:
DependencyGenerator.generate_dependencies(n = 360)

In [7]:
Benchmark.run()

world.test.empty()
11 µs ± 118 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

world.test.new()
36.7 µs ± 463 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

world.test.clone()
16 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

world.test.clone(keep_singletons=True)
19.8 µs ± 955 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


#### Imitating a REALLY big project

Creating **4000** different dependencies, all singletons, in all the different ways that actually impact test utilities.

In [8]:
DependencyGenerator.generate_dependencies(n = 3600)

In [9]:
Benchmark.run()

world.test.empty()
10.8 µs ± 100 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

world.test.new()
36.2 µs ± 342 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

world.test.clone()
68.8 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

world.test.clone(keep_singletons=True)
144 µs ± 19 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Override

In [10]:
x = object()
with world.test.clone():
    %timeit world.test.override.singleton(DependencyGenerator.services[0], x)

2.61 µs ± 13.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
x = object()

def static():
    return x

with world.test.clone():
    %timeit world.test.override.factory(DependencyGenerator.services[0])(static)

3.6 µs ± 31 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
